# Run Predictions

In [1]:
# CTW Prediction
import json
import urllib.request
import os
os.chdir("/home/tales/dev/master/mdc_analysis/")
print("working dir", os.getcwd())

from src.ml.ctw import MyCTW
from src.dao import csv_dao
from src.entity.stop_region import StopRegionGroup, sr_row_to_stop_region
from src.exceptions import exceptions
from src.utils.others import remove_list_elements
from src.exceptions.exceptions import NoCategoryMatched

from src.experiments.ctw_eval import evaluation_ctw_single_partition_light_mem

from src.experiments.markov_eval import evaluation_markov_k_fold_light_mem, evaluation_markov_all_users_vs_one_light_mem, evaluation_markov_single_partition_light_mem

from src.taxonomy.category_mapping import CategoryMapper

from src.exceptions.exceptions import NoCategoryMatched, NotValidTypes

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

working dir /home/tales/dev/master/mdc_analysis


## Loading Data

In [2]:
def get_users_tags_sequence():
    url = "http://127.0.0.1:5000/stop_regions_group"
    response = urllib.request.urlopen(url)
    return json.loads(response.read())

def load_users_tags_sequence():
    with open('outputs/users_tags_sequence.json') as json_file:
        return json.load(json_file)
    
users_tags_sequence = load_users_tags_sequence()

In [3]:
def clean_sequence(sequence):
    new_sequence = []
    
    for tags in sequence:
        if "parking" in tags:
            continue
        else:
            new_sequence.append(tags)

    return new_sequence

In [4]:
categ_mapper = CategoryMapper()

users_categ_sequence = {}
users_categ_sequence_not_list = {}

for user_id in list(users_tags_sequence.keys()):
    print("user_id:", user_id)
    
    users_tags_sequence[user_id] = clean_sequence(users_tags_sequence[user_id])
    
    if len( remove_list_elements(users_tags_sequence[user_id], elements=[[]]) ) < 8:
        continue 
    
    categ_sequence = []

    for tags in users_tags_sequence[user_id]:
        if tags == ["WORK"] or tags == ["HOME"]:
            categ_sequence.append(tags)

        else:
            try:
#                 print("tags:", tags)
                categ = categ_mapper.map_categ(tags, method="most_specific")
                categ_sequence.append([categ])

            except NotValidTypes:
                pass
            
            except NoCategoryMatched:
                categ_sequence.append(["NoCategoryMatched"]) 

    users_categ_sequence[user_id] = categ_sequence
    users_categ_sequence_not_list[user_id] = [categ[0] for categ in categ_sequence]
    

user_id: 6189
user_id: 5936
user_id: 6087
user_id: 5973
user_id: 6085
user_id: 6074
user_id: 6012
user_id: 5982
user_id: 5948
user_id: 5974
user_id: 6090
user_id: 6199
user_id: 6068
user_id: 6024
user_id: 5976
user_id: 6094
user_id: 5941
user_id: 5995
user_id: 5962
user_id: 6093
user_id: 6033
user_id: 6079
user_id: 6038
user_id: 6175
user_id: 6042
user_id: 5924
user_id: 6083
user_id: 6178
user_id: 5958
user_id: 6086
user_id: 6100
user_id: 5945
user_id: 5925
user_id: 5935
user_id: 6172
user_id: 5955
user_id: 6073
user_id: 5980
user_id: 6010
user_id: 5993
user_id: 6037
user_id: 5979
user_id: 5966
user_id: 5985
user_id: 5967
user_id: 5970
user_id: 6169
user_id: 6188
user_id: 6097
user_id: 6096
user_id: 6041
user_id: 6192
user_id: 6064
user_id: 6183
user_id: 5954
user_id: 5972
user_id: 6060
user_id: 6069
user_id: 5937
user_id: 6028
user_id: 5990
user_id: 6071
user_id: 5928
user_id: 5992
user_id: 6058
user_id: 6001
user_id: 5964
user_id: 5950
user_id: 5991
user_id: 6177
user_id: 5961
user_i

In [5]:
# users_tags_sequence = {}

# users = os.listdir("outputs/stop_regions/")
# users.reverse()

# for user_id in users:
#     print("Loading user {} data".format(user_id))
#     users_tags_sequence[user_id] = StopRegionGroup(csv_dao.stop_region_sequence(user_id), 
#                                                     agglutinate_stop_regions=True).sequence_stop_region_tags()["tag"].tolist()

## CTW

#### Same train and test data for each same user

In [10]:
skipped_users = []

depths = [4]

predict_choice_methods = ["random_choice", "most_likely", "random_dummy"]

n=0
for user_id in users_categ_sequence.keys():
    n += 1
    print("n:", n, "-", "user_id:", user_id)
    
    for depth in depths:
        
        for predict_choice_method in predict_choice_methods:
    
            tags_sequence = [categ[0] for categ in users_categ_sequence[user_id]]
            
            if len( remove_list_elements(tags_sequence, elements=[[]]) ) < 8:
                skipped_users.append(user_id)
                continue    

            try:
                evaluation_ctw_single_partition_light_mem(tags_sequence=tags_sequence, 
                                                          user_id=user_id, 
                                                          input_data_version="0.0.categ_v1", 
                                                          predict_choice_method=predict_choice_method,
                                                          dir_name="single_partition/",
                                                          depth=depth,
                                                          repeats_n=3)

            except exceptions.TagsLengthNeedsToBeGreaterThanK:
                print("TagsLengthNeedsToBeGreaterThanK")



n: 1 - user_id: 6189
n: 2 - user_id: 5936
n: 3 - user_id: 6087
n: 4 - user_id: 5973
n: 5 - user_id: 6085
n: 6 - user_id: 6074
n: 7 - user_id: 6012
n: 8 - user_id: 5982
n: 9 - user_id: 5948
n: 10 - user_id: 5974
n: 11 - user_id: 6090
n: 12 - user_id: 6199
n: 13 - user_id: 6068
n: 14 - user_id: 6024
n: 15 - user_id: 5976
n: 16 - user_id: 6094
n: 17 - user_id: 5941
n: 18 - user_id: 5995
n: 19 - user_id: 5962
n: 20 - user_id: 6093
n: 21 - user_id: 6033
n: 22 - user_id: 6079
n: 23 - user_id: 6038
n: 24 - user_id: 6175
n: 25 - user_id: 6042
n: 26 - user_id: 5924
n: 27 - user_id: 6083
n: 28 - user_id: 6178
n: 29 - user_id: 5958
n: 30 - user_id: 6086
n: 31 - user_id: 5945
n: 32 - user_id: 5925
n: 33 - user_id: 5935
n: 34 - user_id: 6172
n: 35 - user_id: 5955
n: 36 - user_id: 6073
n: 37 - user_id: 5980
n: 38 - user_id: 6010
n: 39 - user_id: 5993
n: 40 - user_id: 6037
n: 41 - user_id: 5979
n: 42 - user_id: 5966
n: 43 - user_id: 5985
n: 44 - user_id: 5967
n: 45 - user_id: 5970
n: 46 - user_id: 61

In [ ]:
skipped_users

## Markov

#### Same train and test data for each same user

In [8]:
skipped_users = []

depths = [2]

predict_choice_methods = ["random_choice", "most_likely", "random_dummy"]

n=0
for user_id in users_categ_sequence.keys():
    n += 1
    print("n:", n, "-", "user_id:", user_id)

    if len( remove_list_elements(tags_sequence, elements=[[]]) ) < 8:
        skipped_users.append(user_id)
        continue    

    try:
        evaluation_markov_single_partition_light_mem(tags_sequence=users_categ_sequence[user_id], 
                                                      user_id=user_id, 
                                                      input_data_version="0.0.categ_v1", 
                                                      dir_name="single_partition/",
                                                      repeats_n=3,
                                                      is_distributive=False)
        
    except exceptions.TagsLengthNeedsToBeGreaterThanK:
        print("TagsLengthNeedsToBeGreaterThanK")



n: 1 - user_id: 6189
n: 2 - user_id: 5936
n: 3 - user_id: 6087
n: 4 - user_id: 5973
n: 5 - user_id: 6085
n: 6 - user_id: 6074
n: 7 - user_id: 6012
n: 8 - user_id: 5982
n: 9 - user_id: 5948
n: 10 - user_id: 5974
n: 11 - user_id: 6090
n: 12 - user_id: 6199
n: 13 - user_id: 6068
n: 14 - user_id: 6024
n: 15 - user_id: 5976
n: 16 - user_id: 6094
n: 17 - user_id: 5941
n: 18 - user_id: 5995
n: 19 - user_id: 5962
n: 20 - user_id: 6093
n: 21 - user_id: 6033
n: 22 - user_id: 6079
n: 23 - user_id: 6038
n: 24 - user_id: 6175
n: 25 - user_id: 6042
n: 26 - user_id: 5924
n: 27 - user_id: 6083
n: 28 - user_id: 6178
n: 29 - user_id: 5958
n: 30 - user_id: 6086
n: 31 - user_id: 5945
n: 32 - user_id: 5925
n: 33 - user_id: 5935
n: 34 - user_id: 6172
n: 35 - user_id: 5955
n: 36 - user_id: 6073
n: 37 - user_id: 5980
n: 38 - user_id: 6010
n: 39 - user_id: 5993
n: 40 - user_id: 6037
n: 41 - user_id: 5979
n: 42 - user_id: 5966
n: 43 - user_id: 5985
n: 44 - user_id: 5967
n: 45 - user_id: 5970
n: 46 - user_id: 61

#### Train and Test with the same user

In [ ]:
skipped_users = []

k=4

n=0
for user_id in users_categ_sequence.keys():
    n += 1
    print("n:", n, "-", "user_id:", user_id)

    if len( remove_list_elements(tags_sequence, elements=[[]]) ) < 8:
        skipped_users.append(user_id)
        continue    

    try:
        evaluation_markov_k_fold_light_mem(users_tags_sequence[user_id],
                                           input_data_version="0.0", 
                                           user_id=user_id, 
                                           dir_name="results",
                                           k=k, 
                                           is_distributive=False)
        
        evaluation_markov_k_fold_light_mem(users_tags_sequence[user_id], 
                                           input_data_version="0.0", 
                                           random_dummy_mode="dummy",
                                           user_id=user_id, 
                                           dir_name="results",
                                           k=k, 
                                           is_distributive=False)
        
        evaluation_markov_k_fold_light_mem(users_tags_sequence[user_id], 
                                           input_data_version="0.0.d", 
                                           user_id=user_id, 
                                           dir_name="results",
                                           k=k, 
                                           is_distributive=True)
        
        evaluation_markov_k_fold_light_mem(users_tags_sequence[user_id], 
                                           input_data_version="0.0.d", 
                                           random_dummy_mode="dummy",
                                           user_id=user_id, 
                                           dir_name="results",
                                           k=k, 
                                           is_distributive=True)

        evaluation_markov_k_fold_light_mem(users_categ_sequence[user_id], 
                                           input_data_version="0.0.categ_v1", 
                                           user_id=user_id, 
                                           dir_name="results",
                                           k=k, 
                                           is_distributive=False)

        evaluation_markov_k_fold_light_mem(users_categ_sequence[user_id], 
                                           input_data_version="0.0.categ_v1", 
                                           random_dummy_mode="dummy",
                                           user_id=user_id, 
                                           dir_name="results",
                                           k=k, 
                                           is_distributive=False)


    except exceptions.TagsLengthNeedsToBeGreaterThanK:
        print("TagsLengthNeedsToBeGreaterThanK")



In [ ]:
skipped_users

#### Test with one user, train with all the rest of them

In [ ]:
evaluation_markov_all_users_vs_one_light_mem(users_tags_sequence, 
                                      dir_name="results",
                                      input_data_version="0.0", 
                                      is_distributive=False)

evaluation_markov_all_users_vs_one_light_mem(users_tags_sequence, 
                                      dir_name="results",
                                      random_dummy_mode="dummy", 
                                      input_data_version="0.0", 
                                      is_distributive=False)


evaluation_markov_all_users_vs_one_light_mem(users_tags_sequence, 
                                      dir_name="results",
                                      input_data_version="0.0.d", 
                                      is_distributive=True)

evaluation_markov_all_users_vs_one_light_mem(users_tags_sequence, 
                                      dir_name="results",
                                      random_dummy_mode="dummy", 
                                      input_data_version="0.0.d", 
                                      is_distributive=True)


evaluation_markov_all_users_vs_one_light_mem(users_categ_sequence, 
                                      dir_name="results",
                                      input_data_version="0.0.categ_v1", 
                                      is_distributive=False)

evaluation_markov_all_users_vs_one_light_mem(users_categ_sequence, 
                                      dir_name="results",
                                      random_dummy_mode="dummy", 
                                      input_data_version="0.0.categ_v1", 
                                      is_distributive=False)